In [10]:
import src.models.helpers as mh
import src.models.builders as mb
import src.inference.helpers as ih
import src.data.breathe_data as breathe_data

import pandas as pd

In [3]:
df = breathe_data.load_from_excel()
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125


In [81]:
# Infer AR and IA for all data points given an individuals' age, sex, height, FEV1 and O2 saturation measurements
def infer_AR_IA_for_ID(df):
    df.reset_index(inplace=True)
    _, inf_alg, AR, IA, ecFEV1, O2Sat = mb.point_in_time_model1(df.Height[0], df.Age[0], df.Sex[0])
    # _, inf_alg, AR, IA, ecFEV1, O2Sat = build_model(173, 53, "Male")

    # inf_res = ih.infer(inf_alg, [AR, IA], [[ecFEV1, 1.31], [O2Sat, 97.0]], show_progress=False, joint=False, get_time=True)

    def infer_and_unpack(ecfev1_obs, o2sat_obs):
        res = ih.infer(
            inf_alg,
            [AR, IA],
            [[ecFEV1, ecfev1_obs], [O2Sat, o2sat_obs]],
            show_progress=False,
            joint=False,
            get_time=False,
        )
        # mean_ar = AR.get_mean(res[AR.name].values)
        # mean_ia = IA.get_mean(res[IA.name].values)
        # return mean_ar, mean_ia
        return res[AR.name].values, res[IA.name].values

    res = df.apply(
        lambda row: infer_and_unpack(row["ecFEV1"], row["O2 Saturation"]),
        axis=1,
    )
    return res


resraw = df.groupby("ID").apply(infer_AR_IA_for_ID)
# resraw = df.iloc[np.r_[10:13, 1000:1007]].groupby("ID").apply(infer_AR_IA_for_ID)
res = (
    resraw.apply(pd.Series)
    .reset_index()
    .rename(columns={0: "AR", 1: "IA"})
    .drop(columns="level_1")
)
res

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values


,ID,AR,IA
0,101,"[4.325885612867905e-05, 5.204186795805043e-05,...","[0.7890913910456894, 0.18731794516480818, 0.02..."
1,101,"[2.4125461379186277e-05, 2.9023746531528926e-0...","[0.6220562020672773, 0.29684888196588854, 0.07..."
2,101,"[2.4125461379186277e-05, 2.9023746531528926e-0...","[0.6220562020672773, 0.29684888196588854, 0.07..."
3,101,"[4.325885612867905e-05, 5.204186795805043e-05,...","[0.7890913910456894, 0.18731794516480818, 0.02..."
4,101,"[9.946039499055928e-05, 0.00012030272039134186...","[0.8844314501002895, 0.10899967575332595, 0.00..."
...,...,...,...
20392,358,"[0.12473481300803843, 0.12901333203471843, 0.1...","[0.9008728220064204, 0.09561009435187647, 0.00..."
20393,358,"[0.12473481300803843, 0.12901333203471843, 0.1...","[0.9008728220064204, 0.09561009435187647, 0.00..."
20394,358,"[0.12473481300803844, 0.1290133320347184, 0.12...","[0.7940361825789909, 0.18549455618183203, 0.01..."
20395,358,"[0.11434194793175263, 0.12023910500669459, 0.1...","[0.6088054139470103, 0.3107766601148465, 0.072..."


In [83]:
AR = mh.variableNode("Airway resistance (%)", 0, 90, 2, prior={"type": "uniform"})
IA = mh.variableNode("Inactive alveoli (%)", 0, 30, 1, prior={"type": "uniform"})

res["AR mean"] = res["AR"].apply(lambda x: AR.get_mean(x))
res["IA mean"] = res["IA"].apply(lambda x: IA.get_mean(x))

In [84]:
res

,ID,AR,IA,AR mean,IA mean
0,101,"[4.325885612867905e-05, 5.204186795805043e-05,...","[0.7890913910456894, 0.18731794516480818, 0.02...",58.502814,0.736011
1,101,"[2.4125461379186277e-05, 2.9023746531528926e-0...","[0.6220562020672773, 0.29684888196588854, 0.07...",59.743958,0.968994
2,101,"[2.4125461379186277e-05, 2.9023746531528926e-0...","[0.6220562020672773, 0.29684888196588854, 0.07...",59.743958,0.968994
3,101,"[4.325885612867905e-05, 5.204186795805043e-05,...","[0.7890913910456894, 0.18731794516480818, 0.02...",58.502814,0.736011
4,101,"[9.946039499055928e-05, 0.00012030272039134186...","[0.8844314501002895, 0.10899967575332595, 0.00...",55.679250,0.622275
...,...,...,...,...,...
20392,358,"[0.12473481300803843, 0.12901333203471843, 0.1...","[0.9008728220064204, 0.09561009435187647, 0.00...",8.649044,0.602668
20393,358,"[0.12473481300803843, 0.12901333203471843, 0.1...","[0.9008728220064204, 0.09561009435187647, 0.00...",8.649044,0.602668
20394,358,"[0.12473481300803844, 0.1290133320347184, 0.12...","[0.7940361825789909, 0.18549455618183203, 0.01...",8.649044,0.727311
20395,358,"[0.11434194793175263, 0.12023910500669459, 0.1...","[0.6088054139470103, 0.3107766601148465, 0.072...",9.104899,0.980036


In [85]:
# Save rawres as excel to ../../../../ExcelFiles
res.to_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor.xlsx"
)

# Save arrays as objects
Arrays are currently saved as strings which is painful to process when reading from the file

In [105]:
import pandas as pd
import numpy as np
import src.data.helpers as data_helpers

In [109]:
data_helpers.load_excel("../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx", ["AR", "IA"]).iloc[0].AR

array([4.32588561e-05, 5.20418680e-05, 6.36138198e-05, 7.75790804e-05,
       9.55191830e-05, 1.19697488e-04, 1.49303493e-04, 1.88062011e-04,
       2.41906813e-04, 3.11947196e-04, 4.05305033e-04, 5.32562793e-04,
       7.08000413e-04, 9.52536898e-04, 1.29683793e-03, 1.78572543e-03,
       2.48430929e-03, 3.38772019e-03, 4.41638880e-03, 5.62714540e-03,
       7.34893387e-03, 9.96983212e-03, 1.38611638e-02, 1.95905463e-02,
       2.80335807e-02, 4.02356184e-02, 5.71813601e-02, 7.90824538e-02,
       1.04163749e-01, 1.30829350e-01, 1.48179167e-01, 1.42570513e-01,
       1.05898292e-01, 6.28057462e-02, 2.22243409e-02, 4.69561198e-03,
       3.81251065e-04, 8.98665752e-06, 4.01787876e-08, 6.67901454e-12,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])

In [101]:
df = pd.read_excel("../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx")

In [103]:
# Convert back to arrays
def str_to_array(s):
    s_cleaned = s.replace('\\n', '')
    return np.fromstring(s_cleaned[1:-1], sep=' ')

df["AR"] = df.AR.apply(str_to_array)
df["IA"] = df.IA.apply(str_to_array)

In [104]:
df.iloc[0].AR

array([4.32588561e-05, 5.20418680e-05, 6.36138198e-05, 7.75790804e-05,
       9.55191830e-05, 1.19697488e-04, 1.49303493e-04, 1.88062011e-04,
       2.41906813e-04, 3.11947196e-04, 4.05305033e-04, 5.32562793e-04,
       7.08000413e-04, 9.52536898e-04, 1.29683793e-03, 1.78572543e-03,
       2.48430929e-03, 3.38772019e-03, 4.41638880e-03, 5.62714540e-03,
       7.34893387e-03, 9.96983212e-03, 1.38611638e-02, 1.95905463e-02,
       2.80335807e-02, 4.02356184e-02, 5.71813601e-02, 7.90824538e-02,
       1.04163749e-01, 1.30829350e-01, 1.48179167e-01, 1.42570513e-01,
       1.05898292e-01, 6.28057462e-02, 2.22243409e-02, 4.69561198e-03,
       3.81251065e-04, 8.98665752e-06, 4.01787876e-08, 6.67901454e-12,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])

In [100]:
df.to_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx", index=False
)

In [58]:
df = pd.read_excel("../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor.xlsx")

In [71]:
df.AR.apply(json.loads)[0]

'[4.32588561e-05 5.20418680e-05 6.36138198e-05 7.75790804e-05\n 9.55191830e-05 1.19697488e-04 1.49303493e-04 1.88062011e-04\n 2.41906813e-04 3.11947196e-04 4.05305033e-04 5.32562793e-04\n 7.08000413e-04 9.52536898e-04 1.29683793e-03 1.78572543e-03\n 2.48430929e-03 3.38772019e-03 4.41638880e-03 5.62714540e-03\n 7.34893387e-03 9.96983212e-03 1.38611638e-02 1.95905463e-02\n 2.80335807e-02 4.02356184e-02 5.71813601e-02 7.90824538e-02\n 1.04163749e-01 1.30829350e-01 1.48179167e-01 1.42570513e-01\n 1.05898292e-01 6.28057462e-02 2.22243409e-02 4.69561198e-03\n 3.81251065e-04 8.98665752e-06 4.01787876e-08 6.67901454e-12\n 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00\n 0.00000000e+00]'